## Imports

In [57]:
import readData
import first
import pandas as pd
import statsmodels.formula.api as smf

## Predikcija tezine deteta
### Data mining je pokazao da su sledece promenljive u najvecoj korelaciji sa ciljnom promenljivom "totalwgt_kg"

totalwgt_lb 1.0
totalwgt_kg 1.0
birthwgt_lb 0.94980467921 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.29677557587 LOW BIRTHWEIGHT - BABY 1
prglngth 0.124457431481 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.116741311338 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
mosgest 0.0216074471734 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
babysex 0.0132693426178 BD-2 SEX OF 1ST LIVEBORN BABY FROM THIS PREGNANCY
race_r 0.0124701016314 RACE
race 0.0124701016314 RACE
paydu 0.0117008121449 IB-10 CURRENT LIVING QUARTERS OWNED/RENTED, ETC
rmarout03 0.0111530980604 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 3RD
rmarout6 0.0102049393319 INFORMAL MARITAL STATUS AT PREGNANCY OUTCOME - 6 CATEGORIES
nbrnaliv 0.0100469976218 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
totincr 0.00991250764533 TOTAL INCOME OF R'S FAMILY
fmarcon5 0.00978480805552 FORMAL MARITAL STATUS AT CONCEPTION - 5 CATEGORIES
anynurse 0.00977733797509 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
marout03 0.00938657718235 FORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 3RD
rmarout01 0.00934827776446 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 1ST
marout01 0.00925000423337 FORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 1ST
fmarout5 0.00923513646347 FORMAL MARITAL STATUS AT PREGNANCY OUTCOME
marcon03 0.00919235339658 FORMAL MARITAL STATUS WHEN PREGNANCY BEGAN - 3RD
cebow 0.00913605475556 NUMBER OF CHILDREN BORN OUT OF WEDLOCK
ageprg03 0.00876593611357 AGE WHEN PREGNANCY ENDED - 3RD
pmarpreg 0.0085917477251 WHETHER PREGNANCY ENDED BEFORE R'S 1ST MARRIAGE (PREMARITALLY)
b1premar 0.00842277473568 WHETHER R'S FIRST BIRTH WAS PREMARITAL
mar1bir1 0.00835054554265 MONTHS BTW/1ST MARRIAGE & 1ST BIRTH (OR INTERVIEW)
agecon03 0.00823395478373 AGE WHEN PREGNANCY BEGAN - 3RD
ageprg02 0.00814008906176 AGE WHEN PREGNANCY ENDED - 2ND
agecon02 0.00800637677066 AGE WHEN PREGNANCY BEGAN - 2ND

##### Ucitavanje i spajanje podataka

In [4]:
df = readData.ReadFemPreg()
df['live_outcome'] = (df.outcome == 1).astype(int)
training_set = readData.GetTrainingData(df)
test_set = readData.GetTestData(df)

#live = live[live.prglngth>30]
resp = readData.ReadFemResp()
resp.index = resp.caseid
join = df.join(resp, on='caseid', rsuffix='_r')


## Multi regression

In [5]:
formula = ('totalwgt_kg ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_kg   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     92.14
Date:                Mon, 17 Apr 2017   Prob (F-statistic):          2.48e-130
Time:                        20:57:15   Log-Likelihood:                -8461.5
No. Observations:                9038   AIC:                         1.694e+04
Df Residuals:                    9030   BIC:                         1.700e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
Intercept                2.9413      0.032     92.048      0.000         2.879     3.004
C(race)[T.2]             0.1913      0.016     12.255      0.000         0.161     0.222
C(race)[T.3]             0.1540      0.025      6.065      0.000         0.104     0.204
babysex == 1[T.True]     0.1345      0.013     10.353      0.000         0.109     0.160
nbrnaliv > 1[T.True]    -0.7566      0.049    -15.338      0.000        -0.853    -0.660
paydu == 1[T.True]       0.0541      0.015      3.537      0.000         0.024     0.084
agepreg                  0.0032      0.001      2.525      0.012         0.001     0.006
totincr                  0.0068      0.002      3.530      0.000         0.003     0.011
==============================================================================
Omnibus:                      986.934   Durbin-Watson:                   1.633
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3348.110
Skew:                          -0.544   Prob(JB):                         0.00
Kurtosis:                       5.776   Cond. No.                         205.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Lasso regression

In [30]:
alpha=1e-5
maxiter = 1000

formula = ('totalwgt_kg ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit_regularized(method='coord_descent',
                                    maxiter=maxiter, alpha=alpha, L1_wt=1, start_params=None, cnvrg_tol=1e-08, zero_tol=1e-08)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_kg   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     92.14
Date:                Mon, 17 Apr 2017   Prob (F-statistic):          2.48e-130
Time:                        21:03:50   Log-Likelihood:                -8461.5
No. Observations:                9038   AIC:                         1.694e+04
Df Residuals:                    9030   BIC:                         1.700e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
Intercept                2.9410      0.032     92.041      0.000         2.878     3.004
C(race)[T.2]             0.1912      0.016     12.252      0.000         0.161     0.222
C(race)[T.3]             0.1539      0.025      6.060      0.000         0.104     0.204
babysex == 1[T.True]     0.1345      0.013     10.352      0.000         0.109     0.160
nbrnaliv > 1[T.True]    -0.7561      0.049    -15.327      0.000        -0.853    -0.659
paydu == 1[T.True]       0.0540      0.015      3.534      0.000         0.024     0.084
agepreg                  0.0032      0.001      2.532      0.011         0.001     0.006
totincr                  0.0069      0.002      3.533      0.000         0.003     0.011
==============================================================================
Omnibus:                      987.007   Durbin-Watson:                   1.633
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3348.406
Skew:                          -0.544   Prob(JB):                         0.00
Kurtosis:                       5.777   Cond. No.                         205.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Ridge regression

In [20]:
maxiter = 50
formula = ('totalwgt_kg ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit_regularized(method='coord_descent',
                                    maxiter=50, alpha=alpha, L1_wt=0.0, start_params=None, cnvrg_tol=1e-08, zero_tol=1e-08)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_kg   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     91.99
Date:                Mon, 17 Apr 2017   Prob (F-statistic):          4.03e-130
Time:                        21:01:49   Log-Likelihood:                -8462.0
No. Observations:                9038   AIC:                         1.694e+04
Df Residuals:                    9030   BIC:                         1.700e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
Intercept                2.9676      0.032     92.869      0.000         2.905     3.030
C(race)[T.2]             0.1938      0.016     12.414      0.000         0.163     0.224
C(race)[T.3]             0.1557      0.025      6.130      0.000         0.106     0.205
babysex == 1[T.True]     0.1347      0.013     10.370      0.000         0.109     0.160
nbrnaliv > 1[T.True]    -0.7546      0.049    -15.296      0.000        -0.851    -0.658
paydu == 1[T.True]       0.0610      0.015      3.994      0.000         0.031     0.091
agepreg                  0.0023      0.001      1.810      0.070        -0.000     0.005
totincr                  0.0059      0.002      3.045      0.002         0.002     0.010
==============================================================================
Omnibus:                      983.694   Durbin-Watson:                   1.633
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3328.742
Skew:                          -0.542   Prob(JB):                         0.00
Kurtosis:                       5.768   Cond. No.                         205.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Predikcija ishoda trudnoce
### Data mining je pokazao da su sledece promenljive u najvecoj korelaciji sa ciljnom promenljivom (live_outcome)

live_outcome 1.0
prgoutcome 0.914591222169 OUTCOME OF PREGNANCY (BASED ON PRIORITY ORDERING)
wksgest 0.905070710687 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
prglngth 0.880806434446 DURATION OF COMPLETED PREGNANCY IN WEEKS
pregend1 0.830564337362 BC-1 HOW PREGNANCY ENDED - 1ST MENTION
mosgest 0.810607158364 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
outcome 0.693749715953 PREGNANCY OUTCOME
hasbabes 0.155517551919 WHETHER R HAS HAD ANY LIVE BIRTHS
outcom01 0.124516416525 OUTCOME OF PREGNANCY - 1ST
abortion 0.101526089822 CAPI-BASED TOTAL # OF INDUCED ABORTIONS
lossnum 0.100735197069 CAPI-BASED TOTAL # OF SPONTANEOUS PREGNANCY LOSSES
outcom02 0.0973697843344 OUTCOME OF PREGNANCY - 2ND
lbpregs 0.0943802074912 CAPI-BASED TOTAL # OF PREGS ENDING IN LIVE BIRTH
parity_r 0.0867318917207 TOTAL NUMBER OF LIVE BIRTHS
parity 0.0867318917207 TOTAL NUMBER OF LIVE BIRTHS
numkdhh 0.0671365376373 NUMBER OF BIO/ADOPT/RELATED/LEGAL CHILDREN UNDER AGE 18 IN HOUSEHOLD
outcom03 0.0636627520923 OUTCOME OF PREGNANCY - 3RD
numfmhh 0.0545241176437 NUMBER OF FAMILY MEMBERS IN HOUSEHOLD
roscnt 0.0503930193068 NUMBER OF HOUSEHOLD MEMBERS BASED ON HH ROSTER
rmarout6 0.0491851898917 INFORMAL MARITAL STATUS AT PREGNANCY OUTCOME - 6 CATEGORIES
fmarout5 0.0430924904016 FORMAL MARITAL STATUS AT PREGNANCY OUTCOME
rostscrn 0.0424096276513 # OF HOUSEHOLD MEMBERS BASED ON SCREENER
cohpbeg 0.0384954877344 EG-18A WAS R LIVING W/FATHER OF PREG AT BEGINNING OF PREG
pmarpreg 0.0366765435175 WHETHER PREGNANCY ENDED BEFORE R'S 1ST MARRIAGE (PREMARITALLY)
oldwantr 0.0348289626024 WANTEDNESS OF PREGNANCY - RESPONDENT - CYCLE 4 VERSION
wantresp 0.0348113648714 WANTEDNESS OF PREGNANCY - RESPONDENT - CYCLE 5 VERSION
rmarout01 0.0335641248227 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 1ST
cohpend 0.0328818523229 EG-18B WAS R LIVING W/FATHER OF PREG WHEN PREG ENDED/BABY WAS BORN
oldwantp 0.0309287475439 WANTEDNESS OF PREG - R'S PARTNER (FATHER OF PREGNANCY) - CYCLE 4 VERSION
wantpart 0.0299433286285 WANTEDNESS OF PREG - R'S PARTNER (FATHER OF PREGNANCY) - CYCLE 5 VERSION

The method determines which solver from scipy.optimize is used, and it can be chosen from among the following strings:
‘newton’ for Newton-Raphson, ‘nm’ for Nelder-Mead
‘bfgs’ for Broyden-Fletcher-Goldfarb-Shanno (BFGS)
‘lbfgs’ for limited-memory BFGS with optional box constraints
‘powell’ for modified Powell’s method
‘cg’ for conjugate gradient
‘ncg’ for Newton-conjugate gradient
‘basinhopping’ for global basin-hopping solver
The explicit arguments in fit are passed to the solver, with the exception of the basin-hopping solver. Each solver has several optional arguments that are not the same across solvers. See the notes section below (or scipy.optimize) for the available arguments and for the list of explicit arguments that the basin-hopping solver supports.

In [54]:
training_join = training_set.join(resp, on='caseid', rsuffix='_r')
test_join = test_set.join(resp, on='caseid', rsuffix = 'r')

In [142]:
formula_outcome = 'live_outcome ~ lossnum + abortion + hasbabes + parity + roscnt + numkdhh'
model = smf.logit(formula_outcome, training_join)
#results = model.fit()
results = model.fit(method = 'bfgs', maxiter=200)
#results = model.fit_regularized(alpha =5, disp=False)
results.summary()

Optimization terminated successfully.
         Current function value: 0.435135
         Iterations: 47
         Function evaluations: 49
         Gradient evaluations: 49


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           live_outcome   No. Observations:                10800
Model:                          Logit   Df Residuals:                    10793
Method:                           MLE   Df Model:                            6
Date:                Tue, 18 Apr 2017   Pseudo R-squ.:                  0.3070
Time:                        00:42:57   Log-Likelihood:                -4699.5
converged:                       True   LL-Null:                       -6781.4
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      4.8044     19.297      0.249      0.803       -33.018    42.627
lossnum       -0.8313      0.027    -31.131      0.000        -0.884    -0.779
abortion      -0.7760      0.027    -28.994      0.000        -0.829    -0.724
hasbabes      -3.7398     19.297     -0.194      0.846       -41.562    34.082
parity         0.4178      0.029     14.511      0.000         0.361     0.474
roscnt        -0.0034      0.032     -0.107      0.915        -0.065     0.058
numkdhh        0.0170      0.041      0.410      0.682        -0.064     0.098
==============================================================================
"""

In [143]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

actual = endog['live_outcome']
baseline = actual.mean()
print('baseline: ', str(baseline))

predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
print('sum(true_pos): ',str(sum(true_pos)), 'sum(true_neg):', str(sum(true_neg)))

acc = (sum(true_pos) + sum(true_neg)) / len(actual)
print('acc:', str(acc))

baseline:  0.6786111111111112
sum(true_pos):  6755.0 sum(true_neg): 1792.0
acc: 0.791388888889


In [99]:
predict = (results.predict(test_join) >= 0.5)
predict_df = pd.DataFrame(predict, index = predict[:])

print(predict_df[0].value_counts())

actual = test_join['outcome'] == 1
print(actual.value_counts())
#true_pos = predict * actual
#true_neg = (1 - predict) * (1 - actual)
#sum(true_pos), sum(true_neg)

True     2075
False     629
Name: 0, dtype: int64
2704


True     1818
False     900
Name: outcome, dtype: int64

In [80]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

1.3200000000000001

## Ishod trudnoce Ridge

In [102]:
results = smf.ols(formula_outcome, data=training_join).fit_regularized(method='coord_descent',
                                    maxiter=50, alpha=alpha, L1_wt=0.0, start_params=None, cnvrg_tol=1e-08, zero_tol=1e-08)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           live_outcome   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.321
Method:                 Least Squares   F-statistic:                     853.4
Date:                Tue, 18 Apr 2017   Prob (F-statistic):               0.00
Time:                        00:19:37   Log-Likelihood:                -5004.7
No. Observations:               10800   AIC:                         1.002e+04
Df Residuals:                   10793   BIC:                         1.007e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.8525      0.014     58.820      0.000         0.824     0.881
lossnum       -0.1118      0.003    -37.650      0.000        -0.118    -0.106
abortion      -0.1155      0.003    -34.736      0.000        -0.122    -0.109
hasbabes      -0.1186      0.004    -27.657      0.000        -0.127    -0.110
parity         0.0438      0.003     13.837      0.000         0.038     0.050
roscnt        -0.0022      0.004     -0.509      0.611        -0.011     0.006
numkdhh        0.0130      0.006      2.332      0.020         0.002     0.024
==============================================================================
Omnibus:                      789.041   Durbin-Watson:                   2.233
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              957.249
Skew:                          -0.721   Prob(JB):                    1.37e-208
Kurtosis:                       2.781   Cond. No.                         23.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [109]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

actual = endog['live_outcome']
baseline = actual.mean()
print('baseline: ', str(baseline))

predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
print('sum(true_pos): ',str(sum(true_pos)), 'sum(true_neg):', str(sum(true_neg)))

acc = (sum(true_pos) + sum(true_neg)) / len(actual)
print('acc:', str(acc))

baseline:  0.6786111111111112
sum(true_pos):  7004.0 sum(true_neg): 1383.0
acc: 0.776574074074


## Ishod trudnoce Lasso

In [119]:
results = smf.ols(formula_outcome, data=training_join).fit_regularized(method='coord_descent',
                                    maxiter=1000, alpha=0.005, L1_wt=1.0, start_params=None, cnvrg_tol=1e-08, zero_tol=1e-08)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           live_outcome   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     844.6
Date:                Tue, 18 Apr 2017   Prob (F-statistic):               0.00
Time:                        00:27:16   Log-Likelihood:                -5022.6
No. Observations:               10800   AIC:                         1.006e+04
Df Residuals:                   10793   BIC:                         1.011e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.7780      0.015     53.591      0.000         0.750     0.806
lossnum       -0.1066      0.003    -35.835      0.000        -0.112    -0.101
abortion      -0.1096      0.003    -32.911      0.000        -0.116    -0.103
hasbabes      -0.1029      0.004    -23.968      0.000        -0.111    -0.095
parity         0.0495      0.003     15.620      0.000         0.043     0.056
roscnt         0.0093      0.004      2.116      0.034         0.001     0.018
numkdhh        0.0047      0.006      0.838      0.402        -0.006     0.016
==============================================================================
Omnibus:                      831.853   Durbin-Watson:                   2.238
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              989.449
Skew:                          -0.724   Prob(JB):                    1.39e-215
Kurtosis:                       2.683   Cond. No.                         23.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [115]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

actual = endog['live_outcome']
baseline = actual.mean()
print('baseline: ', str(baseline))

predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
print('sum(true_pos): ',str(sum(true_pos)), 'sum(true_neg):', str(sum(true_neg)))

acc = (sum(true_pos) + sum(true_neg)) / len(actual)
print('acc:', str(acc))

baseline:  0.6786111111111112
sum(true_pos):  7008.0 sum(true_neg): 1377.0
acc: 0.776388888889
